In [1]:
# This notebook demonstrates how to calculate the transofmration matrix by 3D Zernike moments using ZMPY3D with NumPy.
#
# This notebook primarily consists of the following steps: 
#     1. Install ZMPY3D and py3Dmol (visualisation).
#     2. Download example PDB data with coordinates.
#     3. Define necessary parameters.
#     4. Load precalculated cache.
#     5. Convert coordinate data into a voxel.
#     6. Create a callable function for generating Zernike moments and normalization.
#     7. Obtain the transform matrix.
#     8. A command line interface (CLI) example

In [2]:
# # Install ZMPY3D versions for NumPy.
# # Install py3Dmol for visualisation.

! uv pip install ZMPY3D
! uv pip install py3Dmol


# print(f"It is recommended to restart the Python kernel for the IPython notebook.")

Audited 1 package in 1ms
Audited 1 package in 2ms


In [3]:
# # Download example data from GitHub using curl
! curl -OJL https://github.com/tawssie/ZMPY3D/raw/main/6NT5.pdb
! curl -OJL https://github.com/tawssie/ZMPY3D/raw/main/6NT6.pdb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  401k  100  401k    0     0   595k      0 --:--:-- --:--:-- --:--:-- 5990k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  378k  100  378k    0     0   644k      0 --:--:-- --:--:-- --:--:--  644k


In [4]:
import ZMPY3D as z
import numpy as np


Param=z.get_global_parameter()
ResidueBox=z.get_residue_gaussian_density_cache(Param)
GridWidth = 1.0

# Convert structure data into coordinates
[XYZ_A,AA_NameList_A]=z.get_pdb_xyz_ca('./6NT5.pdb')
# Convert coordinates into voxels using precalculated Gaussian densities
[Voxel3D_A,CornerA]=z.fill_voxel_by_weight_density(XYZ_A,AA_NameList_A,Param['residue_weight_map'],GridWidth,ResidueBox[GridWidth])

print(f"Structure A (6NT5) has been converted into a gridded voxel with the shape {Voxel3D_A.shape}")

[XYZ_B,AA_NameList_B]=z.get_pdb_xyz_ca('./6NT6.pdb')
[Voxel3D_B,CornerB]=z.fill_voxel_by_weight_density(XYZ_B,AA_NameList_B,Param['residue_weight_map'],GridWidth,ResidueBox[GridWidth])

print(f"Structure B (6NT6) has been converted into a gridded voxel with the shape {Voxel3D_B.shape}")


Structure A (6NT5) has been converted into a gridded voxel with the shape (80, 78, 102)
Structure B (6NT6) has been converted into a gridded voxel with the shape (92, 57, 100)


In [5]:
import pickle

MaxOrder=6

# Find the cache_data directory based on the site package location of ZMPY3D.
BinomialCacheFilePath=z.__file__.replace('__init__.py', 'cache_data') + '/BinomialCache.pkl'
with open(BinomialCacheFilePath, 'rb') as file:
    BinomialCachePKL = pickle.load(file)

LogCacheFilePath=z.__file__.replace('__init__.py', 'cache_data') + '/LogG_CLMCache_MaxOrder{:02d}.pkl'.format(MaxOrder)

with open(LogCacheFilePath, 'rb') as file:
    CachePKL = pickle.load(file)

BinomialCache=BinomialCachePKL['BinomialCache']

GCache_pqr_linear=CachePKL['GCache_pqr_linear']
GCache_complex=CachePKL['GCache_complex']
GCache_complex_index=CachePKL['GCache_complex_index']
CLMCache3D=CachePKL['CLMCache3D']
CLMCache=CachePKL['CLMCache']
RotationIndex=CachePKL['RotationIndex']

s_id=np.squeeze(RotationIndex['s_id'][0,0])-1
n   =np.squeeze(RotationIndex['n'][0,0])
l   =np.squeeze(RotationIndex['l'][0,0])
m   =np.squeeze(RotationIndex['m'][0,0])
mu  =np.squeeze(RotationIndex['mu'][0,0])
k   =np.squeeze(RotationIndex['k'][0,0])
IsNLM_Value=np.squeeze(RotationIndex['IsNLM_Value'][0,0])-1  



In [6]:

def OneTimeConversion(Voxel3D,Corner,GridWidth,BinomialCache, CLMCache, CLMCache3D, GCache_complex, GCache_complex_index, GCache_pqr_linear, MaxOrder, Param, ResidueBox, RotationIndex):
    
    Dimension_BBox_scaled=Voxel3D.shape;

    XYZ_SampleStruct = {
        'X_sample': np.arange(Dimension_BBox_scaled[0] + 1),
        'Y_sample': np.arange(Dimension_BBox_scaled[1] + 1),
        'Z_sample': np.arange(Dimension_BBox_scaled[2] + 1)
    }

    # Calculate the volume mass and the center of mass
    [VolumeMass,Center,_]=z.calculate_bbox_moment(Voxel3D,1,XYZ_SampleStruct)

    [AverageVoxelDist2Center,MaxVoxelDist2Center]=z.calculate_molecular_radius(Voxel3D,Center,VolumeMass,Param['default_radius_multiplier'])

    Center_scaled=Center*GridWidth+Corner

    ##################################################################################
    # You may add any preprocessing on the voxel before applying the Zernike moment. #
    ##################################################################################

    # Calculate the weights for sphere sampling    
    SphereXYZ_SampleStruct=z.get_bbox_moment_xyz_sample(Center,AverageVoxelDist2Center,Dimension_BBox_scaled)

    # Apply weights to the geometric moments
    _,_,SphereBBoxMoment=z.calculate_bbox_moment(Voxel3D,MaxOrder,SphereXYZ_SampleStruct)

    # Convert to unscaled 3D Zernike moments
    [_, ZMoment_raw]=z.calculate_bbox_moment_2_zm(MaxOrder, GCache_complex, GCache_pqr_linear, GCache_complex_index, CLMCache3D, SphereBBoxMoment)

    # Calculate alternative 3D Zernike moments for specific normalisation orders 2, 3, 4, 5 and 6
    ABList_2=z.calculate_ab_rotation_all(ZMoment_raw, 2)
    ABList_3=z.calculate_ab_rotation_all(ZMoment_raw, 3)
    ABList_4=z.calculate_ab_rotation_all(ZMoment_raw, 4)
    ABList_5=z.calculate_ab_rotation_all(ZMoment_raw, 5)
    ABList_6=z.calculate_ab_rotation_all(ZMoment_raw, 6)

    ABList_all=np.vstack(ABList_2+ABList_3+ABList_4+ABList_5+ABList_6)

    ZMList_all=z.calculate_zm_by_ab_rotation(ZMoment_raw, BinomialCache, ABList_all, MaxOrder, CLMCache,s_id,n,l,m,mu,k,IsNLM_Value)
    
    ZMList_all=np.stack(ZMList_all,axis=3)

    ZMList_all=np.transpose(ZMList_all,(2,1,0,3)) 
    ZMList_all=ZMList_all[~np.isnan(ZMList_all)]
    
    # Based on ABList_all, it is known in advance that Order 6 will definitely have 96 pairs of AB, which means 96 vectors.
    ZMList_all=np.reshape(ZMList_all,(np.int64(ZMList_all.size/96),96)) 

    return Center_scaled, ABList_all,ZMList_all


print(f"Merge all steps into a single callable function, OneTimeConversion, generating alternative 3D Zernike moments for rotational variations")
 

Merge all steps into a single callable function, OneTimeConversion, generating alternative 3D Zernike moments for rotational variations


In [7]:
%%time
# Compute all possible rotations
Center_scaled_A,ABList_A,ZMList_A =OneTimeConversion(Voxel3D_A,CornerA,1.00,BinomialCache, CLMCache, CLMCache3D, GCache_complex, GCache_complex_index, GCache_pqr_linear, MaxOrder, Param, ResidueBox, RotationIndex)
Center_scaled_B,ABList_B,ZMList_B =OneTimeConversion(Voxel3D_B,CornerB,1.00,BinomialCache, CLMCache, CLMCache3D, GCache_complex, GCache_complex_index, GCache_pqr_linear, MaxOrder, Param, ResidueBox, RotationIndex)

# Compare all Zernike moments and select the maximum value using a dot product calculation
M = np.abs(ZMList_A.conj().T @ ZMList_B) # square matrix A^T*B 
MaxValueIndex = np.where(M == np.max(M)) # MaxValueIndex is a tuple that contains an nd array.

i, j = MaxValueIndex[0][0], MaxValueIndex[1][0]

# Compute the transformation matrix for protein structure A
RotM_A=z.get_transform_matrix_from_ab_list(ABList_A[i,0],ABList_A[i,1],Center_scaled_A)
RotM_B=z.get_transform_matrix_from_ab_list(ABList_B[j,0],ABList_B[j,1],Center_scaled_B)
TargetRotM = np.linalg.solve(RotM_B, RotM_A)


print(f"Here is the transformation matrix and computation time provided by ZMPY3D for protein structure A.")
print(TargetRotM)


Here is the transformation matrix and computation time provided by ZMPY3D for protein structure A.
[[-9.57247145e-01  2.89270525e-01  6.83646351e-04  1.29406833e+02]
 [-2.89271305e-01 -9.57245413e-01 -1.82504821e-03  1.80692290e+02]
 [ 1.26484681e-04 -1.94478146e-03  9.99998101e-01 -1.63715984e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
CPU times: user 467 ms, sys: 13.3 ms, total: 480 ms
Wall time: 42.3 ms


In [8]:

# "SetPDB_XYZ_RotM is not a standard function; it merely applies a matrix to atom coordinates.
# It is recommended to use BioPython for this purpose."
z.set_pdb_xyz_rot('./6NT5.pdb',TargetRotM,'6NT5_trans.pdb')

print(f"Apply the transformation matrix to all coordinates of protein structure A (6NT5.pdb) and save the modified structure as 6NT5_trans.pdb.")

Apply the transformation matrix to all coordinates of protein structure A (6NT5.pdb) and save the modified structure as 6NT5_trans.pdb.


In [9]:
import py3Dmol

view = py3Dmol.view(width=400, height=400)

def add_model_to_viewer(file_path, model_style):
    with open(file_path, 'r') as file:
        pdb_data = file.read()
    view.addModel(pdb_data, 'pdb')
    view.setStyle({'model': -1}, model_style)

add_model_to_viewer('6NT5.pdb', {'cartoon': {'color': 'spectrum', 'opacity': 0.6}})
add_model_to_viewer('6NT6.pdb', {'cartoon': {'color': 'gray', 'opacity': 0.8}})

view.zoomTo()

print("")
print("Colored 6NT5 (structure A) and gray 6NT6 (structure B) before the superposition.")
view.show()



Colored 6NT5 (structure A) and gray 6NT6 (structure B) before the superposition.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [10]:
import py3Dmol

view = py3Dmol.view(width=400, height=400)

def add_model_to_viewer(file_path, model_style):
    with open(file_path, 'r') as file:
        pdb_data = file.read()
    view.addModel(pdb_data, 'pdb')
    view.setStyle({'model': -1}, model_style)

add_model_to_viewer('6NT5_trans.pdb', {'cartoon': {'color': 'spectrum', 'opacity': 0.6}})
add_model_to_viewer('6NT6.pdb', {'cartoon': {'color': 'gray', 'opacity': 0.8}})

view.zoomTo()

print("")
print("Colored 6NT5_trans (transformed structure A) and gray 6NT6 (structure B) after the superposition.")
view.show()


Colored 6NT5_trans (transformed structure A) and gray 6NT6 (structure B) after the superposition.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [11]:

# Alternatively, use a system call to compute results via CLI
# ./ZMPY3D_CLI_SuperA2B PDB_A PDB_B
! ZMPY3D_CLI_SuperA2B "./6NT5.pdb" "./6NT6.pdb"


the matrix is for A’s coordinates.
[[-9.57247145e-01  2.89270525e-01  6.83646351e-04  1.29406833e+02]
 [-2.89271305e-01 -9.57245413e-01 -1.82504821e-03  1.80692290e+02]
 [ 1.26484681e-04 -1.94478146e-03  9.99998101e-01 -1.63715984e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
